In [ ]:
from ipywidgets import widgets, Layout
from IPython.core.display import HTML as coreHTML
import numpy as np
import pandas as pd

from jeodpp import inter, imap
import interMap, svgUtils, colors, geojsonUtils

In [ ]:
##################################################################################################
# Display data for world countries using BDAP country collection
##################################################################################################
country_codes = ['LV','AL','CH','AT','HR','ES','IS','RO','IT','VA','HU','SE','NL','CZ','DE','FR',
                 'ME','BE','SI','LU','IE','BA','MC','BG','PL','LI','GB','RS','SM','DK','IM',
                 'EE','SK','GR','LT','NO','PT','AD','MK','MT','GI','FI','XK','CY']

country_selected = ['IT', 'CH']

# Generate random values and create a dictionary: key=countrycode, value=random value in [0.0,100.0]
d = dict(zip(country_codes, list(np.random.uniform(size=len(country_codes),low=0.0,high=100.0))))

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(d.items(), columns=['iso2code', 'value'])
df['label'] = df['value']

colorlist = ['#440154', '#482878', '#3e4989', '#31688e', '#26828e', '#1f9e89', '#35b779', '#6ece58', '#b5de2b', '#fde725'] # Viridis


m = interMap.countriesMap(df, code_column='iso2code',
                          codes_selected=country_selected, stroke_selected='red', 
                          colorlist=colorlist, stdevnumber=2.0,
                          #stroke_width=0.5, stroke='#010101', 
                          width='69%', height='550px',
                          center=[43,12], zoom=5, basemap=13)

svg = svgUtils.graduatedLegend(df, code_column='iso2code',
                                  codes_selected=country_selected, stroke_selected='red', 
                                  colorlist=colorlist, stdevnumber=2.0, 
                                  legendtitle='Legend title', legendunits='KTOE per 100K inhabit.',
                                  fontsize=16, width=340, height=530)

outlegend = widgets.Output(layout=Layout(width='360px',height='550px'))
with outlegend:
    display(coreHTML(svg))
widgets.HBox([m,outlegend])

In [ ]:
##################################################################################################
# Display data for custom geojson
##################################################################################################
%run interMap.py
%run svgUtils.py

country_codes = ['LV','AL','CH','AT','HR','ES','IS','RO','IT','VA','HU','SE','NL','CZ','DE','FR',
                 'ME','BE','SI','LU','IE','BA','MC','BG','PL','LI','GB','RS','SM','DK','IM',
                 'EE','SK','GR','LT','NO','PT','AD','MK','MT','GI','FI','XK','CY']

country_selected = ['IT', 'CH']

# Generate random values and create a dictionary: key=countrycode, value=random value in [0.0,100.0]
d = dict(zip(country_codes, list(np.random.uniform(size=len(country_codes),low=0.0,high=100.0))))

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(d.items(), columns=['iso2code', 'value'])
df['label'] = df['value']

colorlist = ['#440154', '#482878', '#3e4989', '#31688e', '#26828e', '#1f9e89', '#35b779', '#6ece58', '#b5de2b', '#fde725'] # Viridis

m = geojsonMap(df, './data/ne_50m_admin_0_countries.geojson', 'ISO_A2_EH',
                        code_column='iso2code',
                        codes_selected=country_selected, stroke_selected='red', 
                        colorlist=colorlist, stdevnumber=2.0,
                        stroke_width=1.0, stroke='#010101', 
                        width='69%', height='550px',
                        center=[43,12], zoom=5, basemap=13)

svg = graduatedLegend(df, code_column='iso2code',
                      codes_selected=country_selected, stroke_selected='red', 
                      colorlist=colorlist, stdevnumber=2.0, 
                      legendtitle='Legend title', legendunits='KTOE per 100K inhabit.',
                      fontsize=16, width=340, height=530)

outlegend = widgets.Output(layout=Layout(width='360px',height='550px'))
with outlegend:
    display(coreHTML(svg))
widgets.HBox([m,outlegend])

In [ ]:
##################################################################################################
# Display a custom geojson file and create a graduated legend
##################################################################################################
country_codes = ['LV','AL','CH','AT','HR','ES','IS','RO','IT','VA','HU','SE','NL','CZ','DE','FR',
                 'ME','BE','SI','LU','IE','BA','MC','BG','PL','LI','GB','RS','SM','DK','IM',
                 'EE','SK','GR','LT','NO','PT','AD','MK','MT','FI','XK','CY','TR']
print('Input codes:  ',len(country_codes))

# Generate random values and create a dictionary: key=countrycode, value=random value in [0.0,100.0]
d = dict(zip(country_codes,list(np.random.uniform(size=len(country_codes),low=0.0,high=100.0))))

geojson = geojsonUtils.geojsonLoadFile('./data/ne_50m_admin_0_countries.geojson')
geojsonnew = geojsonUtils.geojsonJoin(geojson,'ISO_A2_EH', 'value', d, innerMode=True)

# Verify that all the countries have been joined
joined_codes = geojsonUtils.geojsonAll(geojsonnew,'ISO_A2_EH')
for c in country_codes:
    if not c in joined_codes:
        print(c, 'not found')

j = geojsonUtils.geojsonJson(geojsonnew)
print('Joined codes: ', len(j['features']))
print('Joined names: ', geojsonUtils.geojsonAll(geojsonnew,'NAME'))

vector = interMap.interGeojsonToVector(geojsonnew)

vector.colorScheme('invReds_soft')
vector.legendSet('line', 'stroke-width', '1.0')
vector.legendSet('line', 'stroke',       '#010101')
#vector.legendCategories('NAME')
vector.legendGraduated("value","quantile",8)

vector.parameter("identifyseparator","<br>")
vector.parameter("identifyfield","FID NAME value")

# Add the VectorLayer to the map
p = vector.process()
m = imap.Map(basemap=1, layout=Layout(width='79%',height='600px'))
layer = m.addLayer(p.toLayer())
extents = m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

outlegend = widgets.Output(layout=Layout(width='20%',height='600px'))
with outlegend:
    display(coreHTML(vector.legend("Legend on value attribute", 1)))

widgets.HBox([m,outlegend])

In [ ]:
##################################################################################################
# Load a custom geojson file, create a CUSTOM classification and display on the map
##################################################################################################
geojson = geojsonUtils.geojsonLoadFile('./data/example.geojson')

lid = geojsonUtils.geojsonAll(geojson,'id')  # List of unique id of the polygons
lha = geojsonUtils.geojsonAll(geojson,'ha')  # list of polygon areas in hectares
minha = min(lha)
maxha = max(lha)

# Create equal interval classes on the polygons area
numClasses = 8
rangeha = np.linspace(minha, maxha, num=numClasses)

# Return a class in [0,numClasses]
def ha2Class(h):
    try:
        return next(x[0] for x in enumerate(rangeha) if x[1] >= h)
    except:
        return len(rangeha)

# List of classes for each of the polygons
lclass = [ha2Class(x) for x in lha]

# Dictionary id -> class
id2class = dict(zip(lid,lclass))

# Add 'class' attribute to the geojson
geojsonnew = geojsonUtils.geojsonJoin(geojson, 'id', 'class', id2class)

# Create a VectorLayer instance from the geojsonnew string
vector = interMap.interGeojsonToVector(geojsonnew)

# Create a legend based on class attribute
listOfClasses = list(range(numClasses+1))
scalemin = listOfClasses[0]
scalemax = listOfClasses[-1]
breaks   = listOfClasses[1:]
descriptions = ['Class %d' % x for x in breaks]

vector.colorScheme('invReds_soft')
vector.legendSet('line', 'stroke-width', '1.5')
vector.legendSet('line', 'stroke',       '#010101')
vector.legendGraduated('class', 'custom', len(breaks), 2, scalemin, scalemax, breaks)

m = imap.Map(basemap=1, layout=Layout(width='79%',height='600px'))
outlegend = widgets.Output(layout=Layout(width='20%',height='600px'))

# Add the VectorLayer to the map
p = vector.process()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

# Display the legend inside the Output widget
outlegend.clear_output()
with outlegend:
    display(HTML(svgUtils.categoriesLegend('Title of the legend', descriptions)))

# Display map and legend
widgets.HBox([m,outlegend])  

In [ ]:
##################################################################################################
# Load a custom geojson file and display all the polygons with the same color
##################################################################################################
geojson = geojsonUtils.geojsonLoadFile('./data/ItalyProvinces.geojson')

# Create a VectorLayer instance from the geojson string
vector = interMap.interGeojsonToVector(geojson)

vector.legendSet('line', 'stroke-width', '1.0')
vector.legendSet('line', 'stroke',       '#010101')
vector.legendSingle("#0000ff")
vector.opacity(255)

# Add the VectorLayer to the map
m = imap.Map(basemap=1, layout=Layout(height='400px'))
p = vector.process()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

# Set the identify
vector.parameter("identifyfield", "DEN_PROV")
inter.identifyPopup(m,p)

m

In [ ]:
##################################################################################################
# Add a custom RASTER file to the map: population of the World
##################################################################################################
coll = inter.ImageCollection("SIMPLE")
p = coll.process()

minvalue =  0.0
maxvalue = 30.0
colorSchema = 'Spectral_soft'

p.parameter("file","/eos/jeodpp/data/base/Population/GLOBAL/WorldPop/VER1-0/Data/VRT/MOSAIC_ppp_prj_2020.vrt")
p.parameter("epsg", "4326")
p.parameter("nodata", "-99999.0")
p.parameter("scalemin", str(minvalue))
p.parameter("scalemax", str(maxvalue))
p.parameter("interpolate", 'nearest')  # 'bilinear'
p.parameter("colorscheme", colorSchema)

display(HTML(inter.colorSchemaLegend(colorSchema, Title="Population density per km2 in 2020", Markers=True, Height=32, MinValue=minvalue, MaxValue=maxvalue, NumPixels=700))) 

m = imap.Map(basemap=1, layout=Layout(height='600px'))
m.addLayer(p.toLayer())

identifyPopup(m,p)

m

In [ ]:
##################################################################################################
# Bivariate choropleth map on italian provinces
##################################################################################################
from ipywidgets import widgets, Layout
from jeodpp import inter, imap
import interMap, geojsonUtils

geojson = geojsonUtils.geojsonLoadFile('./data/ItalyProvinces.geojson')
vector = interMap.interGeojsonToVector(geojson)
vector = vector.parameter("identifyfield", "SIGLA DEN_PROV SHAPE_AREA")
vector = vector.parameter("identifyseparator", "<br>")

m = imap.Map(basemap=1, layout=Layout(height='400px'))

colorlist1 = ['#f3f3f3', '#eac5dd', '#e6a3d0']
colorlist2 = ['#f3f3f3', '#c2f1d5', '#8be2ae']

svg = interMap.bivariateLegend(vector,
                               ["[SIGLA] < 'FE'", "[SIGLA] >= 'FE' and [SIGLA] <= 'PU'", "[SIGLA] > 'PU'"],
                               ["[SHAPE_AREA] < 2500000000", "[SHAPE_AREA] >= 2500000000 and [SHAPE_AREA] <= 4500000000", "[SHAPE_AREA] > 4500000000 and [SHAPE_AREA] <= 7500000000", '[SHAPE_AREA] > 7500000000'],
                               #["[SHAPE_AREA] < 2566597560", "[SHAPE_AREA] >= 2566597560 and [SHAPE_AREA] <= 5566597560", '[SHAPE_AREA] > 5566597560'],
                               colorlist1,
                               colorlist2,
                               title='Example of Bivariate Choropleth Map',
                               title1="Province initials",
                               names1=['< FE', 'in [FE,PU]', '> PU'],
                               title2="Province area",
                               names2=['Small', 'Medium', 'Large', 'XLarge'],
                               fontsize=18,
                               fontweight=500,
                               side=80)

p = vector.process()
m.clear()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

display(HTML(svg))
display(m)

In [ ]:
##################################################################################################
# Bivariate choropleth map on AGRI crop data
##################################################################################################
from ipywidgets import widgets, Layout
from jeodpp import inter, imap
import interMap

m = imap.Map(basemap=60, layout=Layout(height='400px'))
vector = inter.loadLocalVector("DEBY_2019_LandCover.shp")
vector = vector.parameter("identifyfield", "LAU_NAME YEAR AL_PERC PC_PERC PG_PERC")
vector = vector.parameter("identifyseparator", "<br>")

colorlist1 = ['#f3f3f3', '#eac5dd', '#e6a3d0']
colorlist2 = ['#f3f3f3', '#c2f1d5', '#8be2ae']

svg = interMap.bivariateLegend(vector,
                               ["[AL_PERC] < 33", "[AL_PERC] >= 33 and [AL_PERC] <= 66", "[AL_PERC] > 66"],
                               ["[PG_PERC] < 10", "[PG_PERC] >= 10 and [PG_PERC] <= 20", "[PG_PERC] > 20"],
                               colorlist1,
                               colorlist2,
                               title='LAU - Bayern Germany',
                               title1="Arable Land",
                               names1=['Low', 'Medium', 'High'],
                               title2="Permanent Grassland",
                               names2=['Low', 'Medium', 'High'],
                               stroke_width=0.5,
                               fontsize=22,
                               fontweight=500,
                               side=100)

p = vector.process()
m.clear()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

display(HTML(svg))
display(m)

In [ ]:
##################################################################################################
# Simple trivariate choropleth map
##################################################################################################
from ipywidgets import widgets, Layout
from jeodpp import inter, imap
import interMap

m = imap.Map(basemap=60, layout=Layout(height='400px'))
v = inter.loadLocalVector("DEBY_2019_LandCover.shp")
v = v.parameter("identifyfield", "LAU_NAME YEAR AL_PERC PC_PERC PG_PERC")
v = v.parameter("identifyseparator", "<br>")

svg = interMap.trivariateLegend(v,
                                "[AL_PERC] > 60",
                                "[PC_PERC] > 10",
                                "[PG_PERC] > 20",
                                color1='#ff60ff',
                                color2='#ffff60',
                                color3='#60ffff',
                                color4='#ffffff55',
                                title='Example of Trivariate Choropleth Map',
                                title1="Arable Land",
                                title2="Perm. Crop",
                                title3="Permanent Grassland",
                                fontsize=15,
                                fontweight=500,
                                radius=100)

v.opacity(255)

m.clear()
m.addLayer(v.toLayer())
m.zoomToImageExtent(v)

inter.identifyPopup(m,v)

display(HTML(svg))
display(m)

In [ ]:
##################################################################################################
# Complex trivariate choropleth map
##################################################################################################
from ipywidgets import widgets, Layout, HTML
from IPython.display import display

from jeodpp import inter, imap
%run interMap.py

# Load data
vector = inter.loadLocalVector("DEBY_2019_LandCover_LAU.shp",impersonateuser="davide")
vector = vector.parameter("identifyfield", "LAU_NAME YEAR AL_PERC PC_PERC PG_PERC")
vector = vector.parameter("identifyseparator", "<br>")

# Create and display a Map instance
m = imap.Map(basemap=60, layout=Layout(height='600px'))
#display(m)

# Creation of the trivariate legend
svg = trivariateLegendEx(vector,
                                  "AL_PERC",
                                  "PC_PERC",
                                  "PG_PERC",
                                  4,
                                  0.0,
                                  100.0,
                                  0.0,
                                  100.0,
                                  0.0,
                                  100.0,
                                  color1='#ff80f7',
                                  color2='#00d1d0',
                                  color3='#cfb000',
                                  color4='#ffffff00',
                                  title='Complex Trivariate Choropleth',
                                  title1="Arable Land",
                                  title2="Perm. Crop",
                                  title3="Perm. Grassl.",
                                  fontsize=18,
                                  fontweight=500,
                                  side=350,
                                  resizewidth='30vw',
                                  resizeheight='40vh',
                                  digits=0,
                                  maxticks=5,
                                  showarrows=True)


# Display of the vector layer on the map
p = vector.process()
m.clear()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

# Display the trivariate choropleth legend
out = widgets.Output(layout=Layout(width='32vw', height='44vh'))
with out:
    display(HTML(svg))
    
display(widgets.HBox([out,m]))

In [ ]:
v = inter.Collection(inter.collections.Vector)

In [ ]:
##################################################################################################
# Complex trivariate choropleth map
##################################################################################################
from ipywidgets import widgets, Layout, HTML
from IPython.display import display

from jeodpp import inter, imap
%run interMap.py

# Load data
vector1 = inter.loadLocalVector("DEBY_2019_LandCover_LAU.shp",impersonateuser="davide")
vector1 = vector1.parameter("identifyfield", "LAU_NAME YEAR AL_PERC PC_PERC PG_PERC")
vector1 = vector1.parameter("identifyseparator", "<br>")

vector2 = inter.Collection(inter.collections.Vector)
vector2.fileLink('/eos/jeodpp/home/users/kanakst/data/LPIS_EU_2020.shp','',4258)

# Create and display a Map instance
m = imap.Map(layout=Layout(height='600px'))
#display(m)

# Creation of the trivariate legend
svg = trivariateLegendEx(vector1,
                                  "AL_PERC",
                                  "PC_PERC",
                                  "PG_PERC",
                                  4,
                                  0.0,
                                  100.0,
                                  0.0,
                                  100.0,
                                  0.0,
                                  100.0,
                                  color1='#ff0000',
                                  color2='#00ff00',
                                  color3='#0000ff',
                                  color4='#ffffff00',
                                  title='Complex Trivariate Choropleth',
                                  title1="Arable Land",
                                  title2="Perm. Crop",
                                  title3="Perm. Grassl.",
                                  fontsize=18,
                                  fontweight=500,
                                  side=350,
                                  resizewidth='30vw',
                                  resizeheight='40vh',
                                  digits=0,
                                  maxticks=5,
                                  showarrows=True)

svg2 = trivariateLegendEx(vector2,
                                  "AL_PERC",
                                  "PC_PERC",
                                  "PG_PERC",
                                  4,
                                  0.0,
                                  100.0,
                                  0.0,
                                  100.0,
                                  0.0,
                                  100.0,
                                  color1='#ff80f7',
                                  color2='#00d1d0',
                                  color3='#cfb000',
                                  color4='#ffffff00',
                                  title='Complex Trivariate Choropleth',
                                  title1="Arable Land",
                                  title2="Perm. Crop",
                                  title3="Perm. Grassl.",
                                  fontsize=18,
                                  fontweight=500,
                                  side=350,
                                  resizewidth='30vw',
                                  resizeheight='40vh',
                                  digits=0,
                                  maxticks=5,
                                  showarrows=True)


# Display of the vector layer on the map
p1 = vector1.process()
p2 = vector2.process()
m.addLayer(p1.toLayer(), name='2019')
m.addLayer(p2.toLayer(), name='2020')
m.zoomToImageExtent(p1)

m.layersControl()

inter.identifyPopup(m,p1)

# Display the trivariate choropleth legend
out = widgets.Output(layout=Layout(width='32vw', height='44vh'))
with out:
    display(HTML(svg))
    
display(widgets.HBox([out,m]))

In [ ]:
##################################################################################################
# Complex trivariate choropleth map
##################################################################################################
from ipywidgets import widgets, Layout, HTML
from IPython.display import display

from jeodpp import inter, imap
import interMap

# Load data
vector1 = inter.loadLocalVector("LPIS_EU_2021_try.shp")

vector2 = inter.VectorLayer('wkt')
vector2.fileLink('/eos/jeodpp/home/users/kanakst/data/LPIS_EU_2020.shp','',4258)
#vector2 = vector2.parameter("identifyfield", "LAU_NAME YEAR_DB AL_perc PC_perc PG_perc AVG_prc_sz no_prc GEOMETRY")
#vector2 = vector2.parameter("identifyseparator", "#")

# Create and display a Map instance
m = imap.Map(layout=Layout(height='600px'))
#display(m)

# Creation of the trivariate legends


svg2 = interMap.trivariateLegendEx(vector2,
                                   "AL_PERC",
                                   "PC_PERC",
                                   "PG_PERC",
                                   4,
                                   0.0,
                                   100.0,
                                   0.0,
                                   100.0,
                                   0.0,
                                   100.0,
                                   color1='#ff80f7',
                                   color2='#00d1d0',
                                   color3='#cfb000',
                                   color4='#ffffff00',
                                   title='Complex Trivariate Choropleth',
                                   title1="Arable Land",
                                   title2="Perm. Crop",
                                   title3="Perm. Grassl.",
                                   fontsize=18,
                                   fontweight=500,
                                   side=350,
                                   resizewidth='30vw',
                                   resizeheight='40vh',
                                   digits=0,
                                   maxticks=5,
                                   showarrows=True)



# Display of the vector layers on the map
p1 = vector1.process()
p2 = vector2.process()
#m.addLayer(p1.toLayer(),name="2021")
m.addLayer(p2.toLayer(),name="2020")
m.zoomToImageExtent(p1)

m.layersControl()

inter.identifyPopup(m,p1)
inter.identifyPopup(m,p2)

# Display the trivariate choropleth legend
out = widgets.Output(layout=Layout(width='32vw', height='44vh'))
with out:
    display(HTML(svg1))
    
display(widgets.HBox([out,m]))

In [ ]:
##################################################################################################
# 2 layers with complex trivariate choropleth legend
##################################################################################################
from ipywidgets import widgets, Layout, HTML
from jeodpp import inter, imap
from vois import interMap

m = imap.Map(layout=Layout(height='600px'))

vector1 = inter.loadLocalVector("LPIS_EU_2021_try.shp")
vector2 = inter.loadLocalVector("LPIS_EU_2020.shp")


# Create the trivariate legend for vector1
svg1 = interMap.trivariateLegendEx(vector1,
                                   "AL_perc",
                                   "PC_perc",
                                   "PG_perc",
                                   4,
                                   0.0,
                                   100.0,
                                   0.0,
                                   100.0,
                                   0.0,
                                   100.0,
                                   color1='#ff80f7',
                                   color2='#00d1d0',
                                   color3='#cfb000',
                                   color4='#ffffff00',
                                   title='Complex Trivariate Choropleth',
                                   title1="Arable Land",
                                   title2="Perm. Crop",
                                   title3="Perm. Grassl.",
                                   fontsize=18,
                                   fontweight=500,
                                   side=350,
                                   resizewidth='30vw',
                                   resizeheight='40vh',
                                   digits=0,
                                   maxticks=5,
                                   showarrows=True)

# Single symbol legend for vector1
#vector1.reset()
#vector1.legendSet('line','stroke','#000000')
#vector1.legendSet('line','stroke-width','0.25')
#vector1.legendSingle("#ff0000")


# Create the trivariate legend for vector2
svg2 = interMap.trivariateLegendEx(vector2,
                                   "AL_perc",
                                   "PC_perc",
                                   "PG_perc",
                                   4,
                                   0.0,
                                   100.0,
                                   0.0,
                                   100.0,
                                   0.0,
                                   100.0,
                                   color1='#ff80f7',
                                   color2='#00d1d0',
                                   color3='#cfb000',
                                   color4='#ffffff00',
                                   title='Complex Trivariate Choropleth',
                                   title1="Arable Land",
                                   title2="Perm. Crop",
                                   title3="Perm. Grassl.",
                                   fontsize=18,
                                   fontweight=500,
                                   side=350,
                                   resizewidth='30vw',
                                   resizeheight='40vh',
                                   digits=0,
                                   maxticks=5,
                                   showarrows=True)


# Single symbol legend for vector2
#vector2.reset()
#vector2.legendSet('line','stroke','#000000')
#vector2.legendSet('line','stroke-width','0.25')
#vector2.legendSingle("#00ff00")


p1 = vector1.process()
p2 = vector2.process()

m.clear()
m.layersControl()

m.addLayer(p1.toLayer(), name="2021")
m.addLayer(p2.toLayer(), name="2020")
m

In [ ]:
display(HTML(svg1))